# Solving Partial Differential Equations in Julia

## JuliaCon 2018

## Chris Rackauckas

## University of Maryland, Baltimore
## University of California, Irvine

## Who this workshop is for

- People who want to learn about and discuss the available tooling to use Julia to build PDE solvers for their complex problem
- People who just want some solution to some specific PDE
- People who know Julia and want to learn what all of the fuss about PDEs is

This talk is for you.

## What is a PDE?

A partial differential equation (PDE) is a differential equation which has partial derivatives. Let's unpack that.

A differential equation describes a value (function) by how it changes. `u' = f(u,p,t)` gives you a solution `u(t)` by you inputing / describing its derivative. Scientific equations are encoded in differential equations since experiments uncovers laws about what happens when entities change.

A partial differential equation describes a value by how it changes in multiple directions: how it changes in the `x` vs `y` physical directions, and how it changes in time.

Thus spatial properties, like the heat in a 3D room at a given time, `u(x,y,z,t)` are described by physical equations which are PDEs. Values like how a drug is distributed throughout the liver, or how stress propogates through an airplane hull, are examples of phonomena described by PDEs.

You are interested in PDEs.

## The best way to solve a PDE is...

## By converting it into another problem!

Generally, PDEs are converted into:

- Linear systems: `Ax = b` find `x`
- Nonlinear systems: `G(x) = 0` find `x`
- ODEs: `u' = f(u,p,t)`, find `u`

## Learning by example: the Poisson Equation

Let's introduce some shorthand: $u_x = \frac{\partial u}{\partial x}$

$$ \Delta u = b $$

In one dimension: $u_{xx} = b$
Two dimensions: $u_{xx} + u_{yy} = b$
Three dimensions: $u_{xx} + u_{yy} + u_{zz} = b$

where $b(x)$, $b(x,y)$, $b(x,y,z)$ is some known constant function (known as "the data")